# MLOps 

This notebook is intended to help you get started managing your machine learning models and deployments in the H2O AI Cloud using python. 

**_This notebook assumes you have run 1 End to End Demo and have a project with a deployed model that exists with a specific name_**

* **Product Documentation:** https://docs.h2o.ai/mlops/
* **Python Documentation:** https://docs.h2o.ai/mlops/py-client-installing/

In [1]:
import os
import requests
import json

import h2o_authn
import h2o_discovery
import h2o_mlops

import pandas as pd

## Connect to MLOps

In [2]:
discovery = h2o_discovery.discover()

token_provider = h2o_authn.TokenProvider(
    refresh_token=os.getenv("H2O_CLOUD_CLIENT_PLATFORM_TOKEN"),
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
)

In [3]:
mlops = h2o_mlops.Client(
    gateway_url=discovery.services['mlops-api'].uri,
    token_provider=token_provider
)

## Projects

### Create a project

In [4]:
project = mlops.projects.create(
    name="my_test_project",
    description="Demo project for a tutorial"
)

### List all projects you have access to

In [5]:
mlops.projects.list()

    | name                    | uid
----+-------------------------+--------------------------------------
  0 | scoring-client-tests    | 0ea7b425-461d-470a-af3c-cef8be604306
  1 | my_test_project         | 73472726-a051-4d72-a8e7-957614c18782
  2 | Telco Churn Predictions | e559825f-b8ba-4812-8191-433df5613c67
  3 | demo                    | f980135b-b38a-4dbd-88f0-f91bc1400eba

### Delete a project 

In [6]:
project.delete()

In [7]:
mlops.projects.list()

    | name                    | uid
----+-------------------------+--------------------------------------
  0 | scoring-client-tests    | 0ea7b425-461d-470a-af3c-cef8be604306
  1 | Telco Churn Predictions | e559825f-b8ba-4812-8191-433df5613c67
  2 | demo                    | f980135b-b38a-4dbd-88f0-f91bc1400eba

### Select a specific project to work with
We have previously created this project using Driverless AI and added models to it in our first tutorial.

In [8]:
project = mlops.projects.list(name="Telco Churn Predictions")[0]

## Experiments

### List experiments
Get a list of all experiments that are in our project

In [9]:
project.experiments.list()

    | name             | uid
----+------------------+--------------------------------------
  0 | Default Baseline | ca85ba4a-a01a-11ee-95af-425f7a7f3fbf

### Select an experiment

In [10]:
experiment = project.experiments.get(uid="ca85ba4a-a01a-11ee-95af-425f7a7f3fbf")

## Models

### List models

Get a list of all models that are in our project

In [11]:
project.models.list()

    | name             | uid
----+------------------+--------------------------------------
  0 | Default Baseline | 7737d493-83cd-46b6-a7e9-3f47008d359e

### Select a model

In [12]:
model = project.models.get(uid="7737d493-83cd-46b6-a7e9-3f47008d359e")

## Deployments

### Deployment environment 
`DEV` and `PROD` are deployment environment tags that you can use for your model deployements. 

In [13]:
environment = project.environments.list(name="DEV")[0]

### Deployment runtime

The scoring runtime is used to load and score a scoring artifact from a model.

In [14]:
runtime = mlops.runtimes.scoring.list(
    artifact_type=model.get_experiment().artifact_types[0],
    uid="dai_mojo_runtime"
)[0]

## Deployment creation

In [15]:
deployment = environment.deployments.create_single(
    name=model.name,
    model=model,
    scoring_runtime=runtime
)

### Deployment health
Wait until our deployment has gone from launching to healthy

In [16]:
import time


while not deployment.is_healthy():
    deployment.raise_for_failure()
    time.sleep(5)

In [17]:
print(deployment.status())
print(deployment.url_for_scoring)
print(deployment.get_sample_request())

HEALTHY
https://model.internal.dedicated.h2o.ai/83250a77-68fd-48bd-8a83-99561fcdc2b8/model/score
{'fields': ['State', 'Account Length', 'Area Code', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls'], 'rows': [['text', '0', '0', 'text', 'text', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']]}


## Make predictions
Using the `requests` library, we make predictions on new data

In [18]:
predictions = requests.post(
    url=deployment.url_for_scoring,
    json=deployment.get_sample_request()
)

In [19]:
predictions.json()

{'fields': ['Churn?.False.', 'Churn?.True.'],
 'id': 'ca85ba4a-a01a-11ee-95af-425f7a7f3fbf',
 'score': [['0.1944492202717737', '0.8055507797282263']]}

In [20]:
pd.DataFrame(predictions.json()["score"], columns=predictions.json()["fields"])

,Churn?.False.,Churn?.True.
0,0.1944492202717737,0.8055507797282263


## Delete a deployment

In [21]:
deployment.delete()